In [ ]:
%%javascript
$('#run_all_cells_below').click()

# Property Based Testing
## (Using Hypothesis)<br><br><br>
### Amsterdam Python Meetup
### 26 April 2017<br><br><br>
### Daniel Bradburn

Property based testing

Choosing properties

Generating data

Model based testing

Django

Examples

say we have a run length encoding function. We encode a string as characters and the number of consecutive occurrences of that character. let's just test this out with something simple

In [ ]:
def encode(input_string):
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [40]:
encode('hellllllllo')

[('h', 1), ('e', 1), ('l', 8), ('o', 1)]

and we also have a decode function which reconstructs the string let's just check this function, let's use the output from the encode

In [ ]:
def decode(lst):
    return ''.join(c * n for c, n in lst)

In [42]:
decode([('h', 1), ('e', 1), ('l', 6), ('o', 1)])

'hellllllo'

but it's probably best to formalize this in a unit test. I'm using pytest here, but you could use unittest or your favourite test runner, the principal is the same.

In [ ]:
def test_run_length_encode():
    input_data = "hello"
    expected = [('h', 1), ('e', 1), ('l', 2), ('o', 1)]
    actual = encode(input_data)
    assert actual == expected

In [43]:
!sh pytest_run.sh test_run_length_encode

.

======================== 21 tests deselected =========================

1 passed, 21 deselected in 0.07 seconds


In [ ]:
def test_run_length_decode():
    input_data = [('h', 1), ('e', 1), ('l', 2), ('o', 1)]
    expected = "hello"
    actual = decode(input_data)
    assert actual == expected

In [44]:
!sh pytest_run.sh test_run_length_decode

.

======================== 21 tests deselected =========================

1 passed, 21 deselected in 0.07 seconds


In [ ]:
import pytest

examples = ['hello', 'python', 'uhm...']

@pytest.mark.parametrize('input_data', examples)
def test_parameterized_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [47]:
!sh pytest_run.sh test_parameterized_run_length_encode_decode

...

======================== 19 tests deselected =========================

3 passed, 19 deselected in 0.08 seconds


In [ ]:
import random, string

random.seed(0)

random_letter = lambda: random.choice(string.ascii_letters)
random_range = lambda m: range(random.randint(0, m))
random_word = lambda m: (random_letter() for i in random_range(m))
random_words = lambda n, m: (''.join(random_word(m)) for n in range(n))

@pytest.mark.parametrize('input_data', random_words(5, 10))
def test_fuzzed_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [46]:
!sh pytest_run.sh test_fuzzed_run_length_encode_decode

.....

======================== 17 tests deselected =========================

5 passed, 17 deselected in 0.08 seconds


In [ ]:
from hypothesis import strategies as st
from hypothesis import given

@given(st.text())
def test_property_based_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [48]:
!sh pytest_run.sh test_property_based_run_length_encode_decode

F.

============================== FAILURES ==============================


____________ test_property_based_run_length_encode_decode ____________


test_1_introduction.py:231: in test_property_based_run_length_encode_decode
    def test_property_based_run_length_encode_decode(input_data):

.venv/hypothesis/core.py:524: in wrapped_test
    print_example=True, is_final=True

.venv/hypothesis/executors.py:58: in default_new_style_executor
    return function(data)

.venv/hypothesis/core.py:111: in run
    return test(*args, **kwargs)

test_1_introduction.py:232: in test_property_based_run_length_encode_decode
    assert decode(encode(input_data)) == input_data

test_1_introduction.py:83: in encode
    lst.append((character, count))
E   UnboundLocalError: local variable 'character' referenced before assignment

----------------------------- Hypothesis -----------------------------

Falsifying example: test_property_based_run_length_encode_decode(input_data='')

======================== 2

In [ ]:
def encode_fixed(input_string):
    count = 1
    prev = ''
    lst = []
    character = ''
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [ ]:
@given(st.text())
def test_property_based_run_length_encode_decode_fixed(input_data):
    assert decode(encode_fixed(input_data)) == input_data

In [49]:
!sh pytest_run.sh test_property_based_run_length_encode_decode_fixed

.

======================== 21 tests deselected =========================

1 passed, 21 deselected in 0.15 seconds


In [ ]:
class Queue(object):

    def __init__(self, max_size):
        self._buffer = [None] * max_size
        self._in, self._out, self.max_size = 0, 0, max_size

    def put(self, item):
        self._buffer[self._in] = item
        self._in = (self._in + 1) % self.max_size

    def get(self):
        result = self._buffer[self._out]
        self._out = (self._out + 1) % self.max_size
        return result

    def __len__(self):
        return (self._in - self._out) % self.max_size

In [ ]:
from hypothesis import strategies as st
from hypothesis.stateful import RuleBasedStateMachine, rule, precondition

class QueueMachine(RuleBasedStateMachine):

    Actual, Model = Queue, list

    def is_created(self): return hasattr(self, 'actual')
    def is_not_empty(self): return self.is_created() and len(self.model)

    @precondition(lambda self: not self.is_created())
    @rule(max_size=st.integers(min_value=1, max_value=10))
    def new(self, max_size):
        self.actual, self.model = self.Actual(max_size), self.Model()
        self.max_size = max_size

    @precondition(is_created)
    @rule(item=st.integers())
    def put(self, item):
        self.actual.put(item)
        self.model.append(item)

    @precondition(is_not_empty)
    @rule()
    def get(self):
        actual, model = self.actual.get(), self.model.pop()
        assert actual == model

    @precondition(is_created)
    @rule()
    def size(self):
        actual, model = len(self.actual), len(self.model)
        assert actual == model

In [50]:
test_model_based_1 = QueueMachine.TestCase
!sh pytest_run.sh test_model_based_1

F

============================== FAILURES ==============================


_____________________ test_model_based_1.runTest _____________________


.venv/hypothesis/stateful.py:182: in runTest
    run_state_machine_as_test(state_machine_class)

.venv/hypothesis/stateful.py:109: in run_state_machine_as_test
    breaker.run(state_machine_factory(), print_steps=True)

.venv/hypothesis/stateful.py:237: in run
    state_machine.execute_step(value)

.venv/hypothesis/stateful.py:512: in execute_step
    result = rule.function(self, **data)

test_4_model_based_testing.py:72: in get
    assert actual == model
E   AssertionError: assert 0 == 1

----------------------------- Hypothesis -----------------------------

Step #1: new(max_size=2)
Step #2: put(item=0)
Step #3: put(item=1)
Step #4: get()

======================== 21 tests deselected =========================

1 failed, 21 deselected in 0.16 seconds


In [37]:
class QueueMachine2(QueueMachine):
            
    @precondition(QueueMachine.is_created)
    @rule(item=st.integers())
    def put(self, item):
        self.actual.put(item)
        self.model.insert(0, item)

In [51]:
test_model_based_2 = QueueMachine2.TestCase
!sh pytest_run.sh test_model_based_2

F

============================== FAILURES ==============================


_____________________ test_model_based_2.runTest _____________________


.venv/hypothesis/stateful.py:182: in runTest
    run_state_machine_as_test(state_machine_class)

.venv/hypothesis/stateful.py:109: in run_state_machine_as_test
    breaker.run(state_machine_factory(), print_steps=True)

.venv/hypothesis/stateful.py:237: in run
    state_machine.execute_step(value)

.venv/hypothesis/stateful.py:512: in execute_step
    result = rule.function(self, **data)

test_4_model_based_testing.py:78: in size
    assert actual == model
E   AssertionError: assert 0 == 2

----------------------------- Hypothesis -----------------------------

Step #1: new(max_size=1)
Step #2: put(item=0)
Step #3: put(item=0)
Step #4: size()

======================== 21 tests deselected =========================

1 failed, 21 deselected in 0.17 seconds


In [55]:
class QueueMachine3(QueueMachine2):

    def is_not_full(self):
        return self.is_created() and len(self.model) < self.max_size

    @precondition(is_not_full)
    @rule(item=st.integers())
    def put(self, item):
        QueueMachine2.put(item)

In [56]:
test_model_based_3 = QueueMachine3.TestCase
!sh pytest_run.sh test_model_based_3

F

============================== FAILURES ==============================


_____________________ test_model_based_3.runTest _____________________


.venv/hypothesis/stateful.py:182: in runTest
    run_state_machine_as_test(state_machine_class)

.venv/hypothesis/stateful.py:109: in run_state_machine_as_test
    breaker.run(state_machine_factory(), print_steps=True)

.venv/hypothesis/stateful.py:237: in run
    state_machine.execute_step(value)

.venv/hypothesis/stateful.py:512: in execute_step
    result = rule.function(self, **data)

test_4_model_based_testing.py:78: in size
    assert actual == model
E   AssertionError: assert 0 == 1

----------------------------- Hypothesis -----------------------------

Step #1: new(max_size=1)
Step #2: put(item=0)
Step #3: size()

======================== 21 tests deselected =========================

1 failed, 21 deselected in 0.15 seconds


In [ ]:
class Queue2(Queue):
    def __init__(self, max_size):
        super(Queue2, self).__init__(max_size + 1)

class QueueMachine4(QueueMachine3):
    Actual = Queue2

In [28]:
test_model_based_4 = QueueMachine4.TestCase
!sh pytest_run.sh test_model_based_4

In [29]:
%%javascript
$('#clear_all_output').click()

<IPython.core.display.Javascript object>

In [31]:
%%HTML
<link href="https://fonts.googleapis.com/css?family=Poppins" rel="stylesheet">
<style>body { font-family: 'Poppins', serif !important; }</style>